# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [2]:
from IPython.display import JSON 
import pandas as pd

In [ ]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [ ]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [3]:
import pandas as pd

In [4]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [5]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [6]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(16862, 6)

### Add the new column with the list of hidden sectors

In [35]:
l=[2,2,-6]
x=2
msvp=[(2,2),(2,-6)]
[xx for (xx,yy) in msvp if xx==x and xx!=yy]

[2]

In [89]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp,SIMPLE=False):
    if not SIMPLE:
        msvp = remove_double_intersection_pairs(msvp,l)
    # ========= Quality check ===========
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return l

def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        msv = [p for p in ps if abs(sum(p))==s]
        lmsv=[x for sublist in msv for x in sublist]
        massives=set(lmsv)        
        massless=set(l).difference(massives)
        if not massless:
            msvp=msv.copy()
            lp=l.copy()
            for msvp in itertools.permutations(msvp.copy()):
                msvp=list(msvp)
                lp=get_real_massless(l,msvp,SIMPLE=True)
                if lp:
                    lp=get_real_massless(l,msvp,SIMPLE=False)
                if not lp:
                    break #found real massless
            if lp:
                continue # Not a hidden sector
            # =================================
            sltn={}
            sltn['S']=s
            #Save storage
            sltn['ψ']=msv
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
# Ana test
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8

In [90]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

CPU times: user 2min 38s, sys: 24 ms, total: 2min 38s
Wall time: 2min 38s


In [91]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [92]:
hs.shape #9466,9233 solutions with hidden sector

(9466, 7)

In [13]:
def get_hidden_sector(l):

    com = list(itertools.combinations_with_replacement(l, 2))
    suma = set([abs(sum(i)) for i in com])

    final = []

    for s in suma:
        cond = [i for i in com if abs(sum(i)) == s]
        aplanar = [item for sublist in cond for item in sublist]
        resta = set(l).difference(set(aplanar))

        if not resta:
            l_ = l.copy()
            N = len(l)

            combs = []

            equal = [par for par in cond if par[0] == par[1]]
            ind_eq = [cond.index(eq) for eq in equal]
            
            for ind in ind_eq:
                cond.pop(ind)

            for par in cond:
                try:
                    for val in par:
                        l_[l_.index(val)] = 0
                    N = N - 2
                    combs.append(par)
                except:
                    pass
                    # print("Discharge pair", par)
            
            for par in equal:
                ind = []
                for val in par:
                    try:
                        ind.append(l_.index(val))
                    except:
                        pass
                for ind_ in ind:
                    l_[ind_] = 0

                if len(set(ind)) == 1:
                    N = N - 1
                elif len(set(ind)) != 1:
                    N = N - len(set(ind))

                combs.append(par)

            if N == 0:
                final.append({"S": s, "ψ": combs})

            del l_, cond

    return final

In [183]:
get_hidden_sector([1, 3, 4, 6, -7, -8, -13, 14])

[]

In [336]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

CPU times: user 1min 29s, sys: 10.6 ms, total: 1min 29s
Wall time: 1min 29s


In [19]:
hs2=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()
hs2.shape

(9095, 7)

In [418]:
hs['ss']  = hs['solution'].apply(str)
hs2['ss'] = hs2['solution'].apply(str)

In [419]:
pd.set_option('display.max_colwidth',200)

In [420]:
tmp=hs2[['ss','hidden']].merge(hs[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [421]:
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )].shape #167

(0, 3)

In [422]:
print( tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:50][["ss",'hidden_1']
].to_json(orient='records',lines=True).replace('\\u03c8','psi').replace('_1','').replace('ss','solution'))

In [177]:
tmp=hs[['ss','hidden']].merge(hs2[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [180]:
print( tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:50][["ss",'hidden_1']
].to_json(orient='records',lines=True).replace('\\u03c8','psi').replace('_1','').replace('ss','solution'))

{"solution":"[1, 3, 4, 6, -7, -8, -13, 14]","hidden":[{"S":7,"psi":[[3,4],[6,-13],[1,6],[-7,14],[1,-8]]}]}
{"solution":"[1, -4, -5, 7, -10, 13, 14, -16]","hidden":[{"S":9,"psi":[[-4,-5],[-4,13],[1,-10],[-5,14],[7,-16]]}]}
{"solution":"[2, 4, 5, 7, -8, -13, -14, 17]","hidden":[{"S":9,"psi":[[5,-14],[4,-13],[4,5],[2,7],[-8,17]]}]}
{"solution":"[3, 5, 5, 7, -9, -15, -15, 19]","hidden":[{"S":10,"psi":[[3,7],[5,-15],[-9,19],[5,5]]}]}
{"solution":"[2, -5, -7, 9, -13, 16, 18, -20]","hidden":[{"S":11,"psi":[[2,-13],[9,-20],[-5,16],[2,9],[-7,18]]}]}
{"solution":"[3, 4, 7, 8, -10, -15, -18, 21]","hidden":[{"S":11,"psi":[[-10,21],[7,-18],[4,7],[3,8],[4,-15]]}]}
{"solution":"[1, -3, -9, 10, -12, 14, 20, -21]","hidden":[{"S":11,"psi":[[10,-21],[1,-12],[-9,20],[-3,14],[1,10]]}]}
{"solution":"[4, 5, 6, 7, -10, -16, -17, 21]","hidden":[{"S":11,"psi":[[5,6],[-10,21],[6,-17],[5,-16],[4,7]]}]}
{"solution":"[1, -7, -7, 11, -13, 19, 19, -23]","hidden":[{"S":12,"psi":[[11,-23],[1,-13],[1,11],[-7,19]]}]}
{"s

In [ ]:
[1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
#Remove [1,6] because is already in the pairs list
[1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
[1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
[1, -7, -8, 14] → [[-7,14],[1,-8]]
[1, -8] → [[1,-8]]
[] → []

In [181]:
get_hidden_sector(   [1, 3, 4, 6, -7, -8, -13, 14] )

[{'S': 7, 'ψ': [(3, 4), (6, -13), (1, 6), (-7, 14), (1, -8)]}]

## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

In [ ]:
pd.set_option('display.max_colwidth',200)

In [ ]:
# What is the minimum n?
hs['n'].min()

7

In [ ]:
# What is the minimum z_max?
hs['zmax'].min()

6

In [ ]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,hidden
2,"[-1, -5, -3]","[-6, -4, -7]","[1, 1, 2, 3, -4, -4, -5, 6]",1,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]"
2295,"[-1, 0, -1, 0, 1]","[-3, -2, -3, -2, 2]","[1, -2, -2, -2, 3, 3, 4, 4, -5, -5, -5, 6]",1,12,6,"[{'S': 7, 'ψ': [(3, 4), (1, 6), (-2, -5)]}]"


In [ ]:
hs[:3]

,l,k,solution,gcd,n,zmax,hidden
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(9, -14), (-10, 15), (5, -10), (3, -8)]}]"
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(-10, 15), (7, -12), (6, -11), (5, -10)]}]"
2,"[-1, -5, -3]","[-6, -4, -7]","[1, 1, 2, 3, -4, -4, -5, 6]",1,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]"


In [ ]:
# How many solutions have several dark sectors?
hs[hs['hidden'].apply(len)>1].shape

(1194, 7)

In [ ]:
hs[hs['hidden'].apply(len)>2].shape

(0, 7)

In [ ]:
# What is the minimum number of DM particles?
hs['hidden'].str[0].str['ψ'].apply(len).min()

3

In [ ]:
hs[hs['hidden'].str[0].str['ψ'].apply(len)==hs['hidden'].str[0].str['ψ'].apply(len).min()][:1]

,l,k,solution,gcd,n,zmax,hidden
4,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,8,"[{'S': 9, 'ψ': [(4, 5), (1, 8), (-2, -7)]}]"


### Same questions for SM solutions

In [ ]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

In [ ]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]
assert get_dark([2,2,2,2,-5,-5,-5,7])==[2, 2, 2, 2, 7]

In [ ]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [ ]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

(138, 7)

In [ ]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(143, 7)

In [ ]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(143, 7)

In [ ]:
hs3['zmax'].min()

8

In [ ]:
hs3[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]"


In [ ]:
hs3[hs3['hidden'].apply(len)>1].shape

(4, 7)

In [ ]:
hs3[hs3['hidden'].apply(len)>2].shape

(2, 7)

In [ ]:
hs3[hs3['hidden'].apply(len)>3].shape

(0, 7)

In [ ]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

2

In [ ]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

,l,k,solution,gcd,n,zmax,hidden
4,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]"
6,"[-1, 0, 1]","[-1, -3, 0, -1]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",1,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]"
8,"[-1, 0, -1, 1]","[-3, 0, -1, 0]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",1,9,15,"[{'S': 11, 'ψ': [(-5, -6), (4, -15)]}]"
16,"[-1, 0, 1]","[-1, -5, 0, 5]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",3,9,20,"[{'S': 17, 'ψ': [(-3, 20), (2, 15)]}]"
18,"[-1, 0, -1, 1]","[-3, 0, -3, 0]","[2, 2, -7, -12, 19, 19, 19, -21, -21]",1,9,21,"[{'S': 19, 'ψ': [(-7, -12), (2, -21)]}]"
19,"[-1, 0, 1]","[-1, 1, 0, -7]","[3, 3, -4, 14, 14, -17, -17, -17, 21]",3,9,21,"[{'S': 17, 'ψ': [(-4, 21), (3, 14)]}]"
24,"[1, 0, 1, -1]","[-3, 0, -3, 0]","[4, 4, -5, 15, 15, -19, -19, -19, 24]",1,9,24,"[{'S': 19, 'ψ': [(4, 15), (-5, 24)]}]"


### Extend the data scheme with number of generations

In [ ]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'MG':mgmax})
    return g
l=[2, 2, -3, 15, 15, -17, -17, -17, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'MG': 0}]

In [ ]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [ ]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [ ]:
hs['Gmax']=-hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [ ]:
hs=hs.sort_values(['n','Gmax']).reset_index(drop=True)

In [ ]:
hs[hs['n']==7]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(9, -14), (-10, 15), (5, -10), (3, -8)]}]","[{'DG': 1, 'MG': 0}]",-1
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(-10, 15), (7, -12), (6, -11), (5, -10)]}]","[{'DG': 1, 'MG': 0}]",-1


In [ ]:
hs[hs['n']==8][:6]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
2,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(2, 2), (-3, 7), (1, -5), (2, -6)]}]","[{'DG': 1, 'MG': 3}]",-4
3,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 6, 'ψ': [(5, -11), (3, -9), (3, 3), (-7, 13)]}]","[{'DG': 1, 'MG': 3}]",-4
4,"[0, -1, -2]","[-1, -6, -5, -6]","[1, -6, -6, -6, 7, 11, 18, -19]",2,8,19,"[{'S': 12, 'ψ': [(-6, -6), (-6, 18), (1, 11), (7, -19)]}]","[{'DG': 1, 'MG': 3}]",-4
5,"[-3, -4, -3]","[-5, 3, -7]","[3, 5, 5, 7, -9, -15, -15, 19]",2,8,19,"[{'S': 10, 'ψ': [(3, 7), (5, -15), (-9, 19), (5, 5)]}]","[{'DG': 2, 'MG': 2}]",-4
6,"[3, 0, -5]","[-1, 1, -1]","[4, 4, 4, 11, -12, -13, -19, 21]",8,8,21,"[{'S': 8, 'ψ': [(11, -19), (-13, 21), (4, 4), (4, -12)]}]","[{'DG': 1, 'MG': 3}]",-4
7,"[0, -2, 0]","[-3, -1, 1, -1]","[5, 5, 5, -15, 19, -21, -29, 31]",2,8,31,"[{'S': 10, 'ψ': [(-21, 31), (5, -15), (19, -29), (5, 5)]}]","[{'DG': 1, 'MG': 3}]",-4


In [ ]:
hs[hs['n']==9][:3]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1761,"[-2, 0, 1, 3]","[-1, 0, -1, 3]","[1, 1, 2, 2, -6, 9, -10, -12, 13]",1,9,13,"[{'S': 3, 'ψ': [(-10, 13), (-6, 9), (1, 2), (9, -12)]}]","[{'DG': 2, 'MG': 0}]",-2
1762,"[1, 2, 0, -1]","[-5, 2, 0, -5]","[3, -8, 10, 10, 11, -15, -15, -16, 20]",1,9,20,"[{'S': 5, 'ψ': [(10, -15), (-15, 20), (11, -16), (3, -8)]}]","[{'DG': 2, 'MG': 0}]",-2
1763,"[-2, 3, 2]","[-3, -8, -2, 1]","[1, 4, -10, -10, -12, 15, 15, 17, -20]",6,9,20,"[{'S': 5, 'ψ': [(-10, 15), (15, -20), (-12, 17), (1, 4)]}]","[{'DG': 2, 'MG': 0}]",-2


In [ ]:
hs[hs['n']==11][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1778,"[-2, -1, -2, -1, 3]","[-3, -1, 2, -1, 3]","[1, 1, 1, 1, -3, 5, -6, 8, -9, -10, 11]",2,11,11,"[{'S': 2, 'ψ': [(1, -3), (-9, 11), (8, -10), (-3, 5), (-6, 8), (1, 1)]}]","[{'DG': 1, 'MG': 4}]",-5
1779,"[-4, 1, 0, 4]","[-1, -3, 1, 0, 2]","[1, 4, 4, 4, 4, -8, -9, 16, -17, -24, 25]",4,11,25,"[{'S': 8, 'ψ': [(1, -9), (-8, 16), (-17, 25), (16, -24), (4, 4)]}]","[{'DG': 1, 'MG': 4}]",-5
1780,"[-1, 0, 1, 0, 2]","[-3, -2, -1, -2, 0]","[1, 1, -2, -2, 4, 4, -6, 7, -8, -9, 10]",2,11,10,"[{'S': 2, 'ψ': [(4, -6), (-8, 10), (-2, 4), (7, -9), (1, 1)]}]","[{'DG': 2, 'MG': 2}]",-4
1781,"[-2, 1, 0, -1]","[-2, -4, -2, 1, 2]","[1, -2, -2, -4, -5, 6, 6, -7, 8, 11, -12]",1,11,12,"[{'S': 4, 'ψ': [(8, -12), (-7, 11), (-2, -2), (-2, 6), (1, -5), (-4, 8)]}]","[{'DG': 2, 'MG': 2}]",-4


In [ ]:
hs[hs['n']==12][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
2295,"[1, 0, -2, 0, -2]","[-3, -1, 1, -1, 1]","[1, 1, 7, 7, 7, 7, 7, -11, -15, -15, -21, 25]",2,12,25,"[{'S': 14, 'ψ': [(1, -15), (7, 7), (-11, 25), (7, -21)]}]","[{'DG': 2, 'MG': 5}]",-7
2296,"[-1, 0, -3, 0, -1]","[-1, 1, -1, -3, -1]","[5, 5, 7, 7, 7, 7, 7, -15, -19, -19, -21, 29]",2,12,29,"[{'S': 14, 'ψ': [(-15, 29), (7, 7), (7, -21), (5, -19)]}]","[{'DG': 2, 'MG': 5}]",-7
2297,"[2, 0, -1, 0, 1]","[-3, -2, -3, -2, -1]","[1, -2, -2, -3, 4, 4, 4, 4, -5, -6, -6, 7]",2,12,7,"[{'S': 8, 'ψ': [(-2, -6), (-3, -5), (4, 4), (1, 7)]}]","[{'DG': 2, 'MG': 4}]",-6
2298,"[-1, 1, -2, -1, 1]","[-2, 0, -1, -3, 0]","[1, 1, 1, 1, 1, 2, -3, -4, 5, -6, -7, 8]",1,12,8,"[{'S': 2, 'ψ': [(1, -3), (2, -4), (-3, 5), (5, -7), (-6, 8), (1, 1)]}]","[{'DG': 1, 'MG': 5}]",-6


In [ ]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [ ]:
hs3[hs3['n']==8][:1]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 1}]",-2


In [ ]:
hs3[hs3['n']==9][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
3,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]","[{'DG': 1, 'MG': 2}]",-3
4,"[1, -2, -1]","[-4, -9, -5, -3]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",6,9,17,"[{'S': 18, 'ψ': [(-9, -9), (1, 17), (-2, -16)]}]","[{'DG': 1, 'MG': 2}]",-3
5,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]","[{'DG': 2, 'MG': 0}]",-2
6,"[2, -2, 1, 3]","[-5, -4, -3, -1]","[1, -2, -2, -2, 5, -7, 8, 9, -10]",1,9,10,"[{'S': 2, 'ψ': [(8, -10), (5, -7), (1, 1), (-7, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [ ]:
hs3[hs3['n']==10][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
44,"[1, 0, -2, -1]","[-3, -1, -3, -5]","[1, 1, -3, -3, -3, 7, -9, 11, 11, -13]",2,10,13,"[{'S': 2, 'ψ': [(-9, 11), (7, -9), (1, 1), (11, -13)]}]","[{'DG': 1, 'MG': 2}]",-3
45,"[-4, -1, 0, 1]","[-4, 1, -1, -3, 1]","[1, 1, 1, 2, 2, -7, 11, -12, -15, 16]",3,10,16,"[{'S': 4, 'ψ': [(-12, 16), (2, 2), (-7, 11), (11, -15)]}]","[{'DG': 1, 'MG': 2}]",-3
46,"[-3, 3, 0, -6]","[-3, -2, 3, 4]","[3, 3, 3, -4, 9, 9, -10, -14, -27, 28]",18,10,28,"[{'S': 18, 'ψ': [(-10, 28), (-4, -14), (9, -27), (9, 9)]}]","[{'DG': 1, 'MG': 2}]",-3
47,"[-1, 0, -4, -3]","[-3, 1, -1, -5]","[1, -3, -3, -3, -9, -9, 11, 17, 27, -29]",2,10,29,"[{'S': 18, 'ψ': [(-9, 27), (11, -29), (1, 17), (-9, -9)]}]","[{'DG': 1, 'MG': 2}]",-3
48,"[0, 1, 0, -2]","[-1, -3, 3, -1, -3]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(-5, -13), (1, -19), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [ ]:
hs3[hs3['n']==11][:2]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
67,"[2, -1, 2, -1]","[-2, -1, -2, 2, 4]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",1,11,14,"[{'S': 18, 'ψ': [(5, 13), (4, 14), (-9, -9), (8, 10)]}]","[{'DG': 1, 'MG': 2}]",-3
68,"[2, 3, 0, -3, 3]","[0, -3, 1, -1, -3]","[1, 1, -2, -3, 6, 6, 6, -7, -10, -10, 12]",15,11,12,"[{'S': 9, 'ψ': [(-3, 12), (1, -10), (-2, -7)]}]","[{'DG': 2, 'MG': 0}]",-2


In [ ]:
hs3[hs3['n']==12][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
86,"[1, -3, -2, 0, -1]","[-3, -1, -3, -1, -3]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",2,12,13,"[{'S': 6, 'ψ': [(1, 5), (3, 3), (-7, 13), (1, -7)]}]","[{'DG': 2, 'MG': 3}]",-5
87,"[1, 0, 2, 0, 2]","[-3, -1, -3, -1, -3]","[7, 9, 9, 9, 11, -13, -13, -27, -27, -27, 31, 31]",2,12,31,"[{'S': 18, 'ψ': [(9, 9), (7, 11), (-13, 31)]}]","[{'DG': 2, 'MG': 3}]",-5
88,"[-2, -3, -2, -3, 2]","[-2, -1, -3, -1, 1]","[1, 1, -2, -2, -2, 6, 6, 7, -8, -8, -9, 10]",4,12,10,"[{'S': 2, 'ψ': [(-8, 10), (6, -8), (7, -9), (1, 1)]}]","[{'DG': 2, 'MG': 2}]",-4
89,"[-3, -1, -2, 3, -3]","[-3, -2, -3, -2, 0]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",19,12,12,"[{'S': 18, 'ψ': [(6, 12), (-9, -9), (-8, -10), (7, 11)]}]","[{'DG': 1, 'MG': 3}]",-4
90,"[-3, -1, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 4, -8, -9, -9, -9, -10, 14, 15, 15, 15, -21]",12,12,21,"[{'S': 18, 'ψ': [(3, -21), (4, 14), (-9, -9), (-8, -10)]}]","[{'DG': 1, 'MG': 3}]",-4
